In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
train=pd.read_csv("../input/ipl-2020-player-performance/Training.csv")

In [4]:
ipl1=pd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2008-2019.csv")
ipl2=pd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2020.csv")

In [5]:
ipl2.head()

In [6]:
ipl2['season'] = '2020'

In [7]:
ipl2.head()

In [8]:
print(train.head())
print("_"*100)
print(ipl1.head())
print("_"*100)
print(ipl2.head())

In [9]:
ipl=pd.concat([ipl1,ipl2])
#concatenating is fine, in this case as see null values will help to filter out the data later

In [10]:
ipl.head()

In [11]:
test=pd.read_csv("../input/ipl-2020-player-performance/sample_submission.csv")

In [12]:
test.head()

In [13]:
test['player_name']=test['Id']
test['match_id']=test['Id']

In [14]:
for i in range(len(test)):
    test['player_name'][i]=test['Id'][i].split('_')[1]

In [15]:
for i in range(len(test)):
    test['match_id'][i]=int(test['Id'][i].split('_')[0])

In [16]:
test=pd.merge(test,ipl,how='inner',on='match_id')

In [17]:
test.head()

In [18]:
test.isnull().sum()

In [19]:
test.drop(['toss_winner', 'toss_decision', 'result',
       'dl_applied', 'winner', 'win_by_runs', 'win_by_wickets',
       'player_of_match','umpire1', 'umpire2', 'umpire3'],axis=1,inplace=True)

In [20]:
test.isnull().sum()

# preparaing training data

In [21]:
train['player_name']=train['Id']
train['match_id']=train['Id']

In [22]:
for i in range(len(train)):
    train['player_name'][i]=train['Id'][i].split('_')[1]

In [23]:
for i in range(len(train)):
    train['match_id'][i]=int(train['Id'][i].split('_')[0])

In [24]:
train.head()

In [25]:
train.drop(['Runs', 'Boundaries', 'Six', 'Fifty', 'Hundred', 'Duck',
       'Batting_Points', 'Wickets', '4W_Haul', '5W_Haul', 'Maidens',
       'Bowling_Points'],axis=1)

In [26]:
train.isnull().sum()

In [27]:
train=pd.merge(train,ipl,how='inner',on='match_id')

In [28]:
train.head()

In [29]:
train.drop(['Runs', 'Boundaries', 'Six', 'Fifty', 'Hundred', 'Duck',
       'Batting_Points', 'Wickets', '4W_Haul', '5W_Haul', 'Maidens',
       'Bowling_Points', 'toss_winner', 'toss_decision', 'result',
       'dl_applied', 'winner', 'win_by_runs', 'win_by_wickets',
       'player_of_match', 'umpire1', 'umpire2', 'umpire3'],axis=1,inplace=True)

In [30]:
print(train.isnull().sum())
print(test.isnull().sum())

In [31]:
train = train.dropna()
test = test.dropna()

In [32]:
train.head()

In [33]:
test.head()

# Label encoding for fitting the model

In [34]:
sar=['season', 'city','date', 'team1', 'team2', 'venue']
for i in sar:
    ord_enc = OrdinalEncoder()
    train[i] = ord_enc.fit_transform(train[[i]])
train.head(11)

In [35]:
sar=['season', 'city','date', 'team1', 'team2', 'venue']
for i in sar:
    ord_enc = OrdinalEncoder()
    test[i] = ord_enc.fit_transform(test[[i]])
test.head(11)

In [36]:
print(train.head())
print("-"*100)
print(train.head())

In [37]:
#scaler = StandardScaler()

In [38]:
X=train.drop(['Total Points','Id','player_name'],axis=1)
y=train['Total Points']

# Random forest regressor

In [39]:
RFR=RandomForestRegressor()
RFR.fit(X,y)

In [40]:
RFR_pred=RFR.predict(X)
np.unique(y_pred)

In [ ]:
print('MSE:',mean_squared_error(y, RFR_pred))
print('r2_Score:',r2_score(y, RFR_pred))
print('RMSE:',np.sqrt(mean_squared_error(y, RFR_pred)))

In [ ]:
X_test=test.drop(['Total Points','Id','player_name'],axis=1)

In [ ]:
testRFR_pred=RFR.predict(X_test)
testRFR_pred

In [ ]:
print('MSE:',mean_squared_error(y[:1283], testRFR_pred))
print('r2_Score:',r2_score(y[:1283], testRFR_pred))
print('RMSE:',np.sqrt(mean_squared_error(y[:1283], testRFR_pred)))

# Decision tree regressor

In [ ]:
DTR = DecisionTreeRegressor()
DTR.fit(X,y)

In [ ]:
testDTR_pred=DTR.predict(X_test)
testDTR_pred

In [ ]:
print('MSE:',mean_squared_error(y[:1283], testDTR_pred))
print('r2_Score:',r2_score(y[:1283], testDTR_pred))
print('RMSE:',np.sqrt(mean_squared_error(y[:1283], testDTR_pred)))

# Support Vector Regressor

In [ ]:
regressor = SVR(kernel = 'rbf')
regressor.fit(X, y)

In [ ]:
testSVR_pred=regressor.predict(X_test)
test_pred

In [ ]:
print('MSE:',mean_squared_error(y[:1283], testSVR_pred))
print('r2_Score:',r2_score(y[:1283], testSVR_pred))
print('RMSE:',np.sqrt(mean_squared_error(y[:1283], testSVR_pred)))

# Submission csv

In [ ]:
submission=test[['Id']]
submission['Total Points']=testRFR_pred[:1283]
submission.head()
submission.to_csv('RFR.csv',index=False)

In [ ]:
submission=test[['Id']]
submission['Total Points']=testDTR_pred[:1283]
submission.head()
submission.to_csv('DTR.csv',index=False)

In [ ]:
submission=test[['Id']]
submission['Total Points']=testSVR_pred[:1283]
submission.head()
submission.to_csv('SVR.csv',index=False)

In [ ]:
#atlast I could have loop to make the process easier, But left that for future improvements :)
#Thank you

# An effort by Sarvesh Kumar Thapa